# Защита персональных данных клиентов страховой компании

Нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Подтвердим корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## Загрузка данных

In [1]:
# импортируем нужные библиотеки, инструменты и метрики
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
# загрузим данные и посмотрим на первые строки
data = pd.read_csv('insurance_info.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
# убедимся, что в датасете нет пропусков
data.isna().sum().sum()

0

In [4]:
# узнаем размер датасета и типы данных в разных столбцах
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
# посмотрим на распределение признаков
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


- Распределение по полу в нашем датасете - примерно 50/50. 
- Возраст варьируется в пределах от 18 до 65 лет, в среднем - 30 лет. 
- Зарплата - от 5 300 руб до 79 000 руб, в среднем около 40 000 руб. 
- Количество членов семьи варьируется от 0 до 6 человек, обычно это 1 человек (не считая клиента). 

In [6]:
# посмотрим на распределение целевого признака
data['Страховые выплаты'].value_counts(normalize = True)

0    0.8872
1    0.0846
2    0.0230
3    0.0036
4    0.0014
5    0.0002
Name: Страховые выплаты, dtype: float64

- Большинство застрахованных (почти 89%) не получали страховых выплат. 
- 8,5% клиентов получали страховую выплату 1 раз. 
- 2,3% - 2 раза. 
- Около 0,5% получали страховые выплаты 3 и более раз.

Данные не нуждаются в дополнительной предобработке. Перейдем к задаче преобразования данных для защиты персональной информации клиентов.

## Метод преобразования данных

Для защиты личных данных клиентов попробуем умножить матрицу признаков на произвольную обратимую матрицу. 

Изменится ли качество модели линейной регрессии после такого преобразования?

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Сравним векторы весов линейной регрессии до преобразования и после:

Вектор весов линейной регрессии **до преобразования:**
$$
w = (X^T X)^{-1} X^T y
$$

Умножим матрицу признаков Х на произвольную квадратную обратимую матрицу Р. 

Чтобы операция умножения матриц была возможна, количество строк / столбцов матрицы Р должно совпадать с шириной матрицы признаков. 

(В данном случае - с учетом нулевого столбца с единицами, который соответствует сдвигу $w_0$.)

Вектор весов линейной регрессии **после преобразования:**
$$
w_p = ((XP)^T XP)^{-1} (XP)^T y 
$$
Упростим формулу:
$$
w_p = (P^T X^T X P)^{-1} P^T X^T y 
$$
$$
w_p = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y 
$$
$$
w_p = P^{-1} (X^T X)^{-1} X^T y 
$$
В результате получаем:
$$
w_p = P^{-1} w
$$

При вычислениях опирались на такие **свойства матриц:**
- операция умножения матриц ассоциативна: $А(ВС) = (AB)C$
- транспонированное произведение матриц равно произведению транспонированных матриц в обратном порядке: $(AB)^T = B^TA^T$
- $(AB)^{-1} = B^{-1}A^{-1}$ для квадратных обратимых матриц

Посмотрим, как будут выглядеть предсказания в обоих случаях (для исходной и преобразованной матриц):

Предсказание **для исходной матрицы признаков:**
$$
a = Xw = X(X^T X)^{-1} X^T y
$$

Предсказание **для преобразованной матрицы признаков:**
$$
a_p = XPw_p = XPP^{-1}w = Xw = a
$$

Таким образом, в обоих случаях получим одинаковые предсказания. 

Передавая данные в преобразованном виде (умножая матрицу признаков на обратимую матрицу соответствующего размера), получим тот же результат, что и без преобразования. 

**Качество модели не изменится.** 

## Алгоритм преобразования

**Алгоритм**

Алгоритм должен принимать данные, преобразовывать их методом умножения на обратимую матрицу соответствующего размера, затем обучать модель на преобразованных данных и выдавать предсказания.

Убедимся в том, что качество моделей, обученных на исходных и на преобразованных данных, одинаковое.

Выделим признаки и целевой признак, а затем разделим наш датасет на обучающую и тестовую выборки:

In [7]:
X = data.drop(['Страховые выплаты'], axis = 1)
X

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0
...,...,...,...,...
4995,0,28.0,35700.0,2
4996,0,34.0,52400.0,1
4997,0,20.0,33900.0,2
4998,1,22.0,32700.0,3


In [8]:
y = data['Страховые выплаты']
y

0       0
1       1
2       0
3       0
4       0
       ..
4995    0
4996    0
4997    0
4998    0
4999    0
Name: Страховые выплаты, Length: 5000, dtype: int64

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 12345)

In [10]:
# проверим, что получили выборки нужного размера
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4000, 4), (1000, 4), (4000,), (1000,))

Преобразуем все выборки в матрицы:

In [11]:
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

Создадим класс модели многомерной линейной регрессии, обучим модель на исходных данных и высчитаем R2 score:

In [12]:
class LinReg:
    
    def fit(self, train_features, train_target):
#добавим столбец с единицами, соответствующий параметру w0:
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
#получим оптимальные параметры модели на обучающей выборке:
        self.w = np.linalg.inv(X.T @ X) @ X.T @ y
 
    def predict(self, test_features):
#добавим столбец с единицами, соответствующий параметру w0:
        X_t = np.concatenate((np.ones((test_features.shape[0], 1)), test_features), axis=1)
#получим предсказания для тестовой выборки:
        return X_t @ self.w
    
model = LinReg()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print(r2_score(y_test, predictions))

0.41176839567704737


Напишем функцию для преобразования обучающей и тестовой выборок с использованием случайной обратимой матрицы:

In [13]:
def matrix_transform(features_train, features_test):
    s = features_train.shape[1]
    for i in range(100):
        P = np.random.normal(size = (s,s))
        if np.linalg.det(P) != 0:
            break
        else:
            continue
    return features_train @ P, features_test @ P

## Проверка алгоритма

**Шаг 1:** Применим функцию для преобразования наших данных.

In [14]:
X_train_t, X_test_t = matrix_transform(X_train, X_test) 

**Шаг 2:** Обучим модель и получим предсказания на преобразованных данных. Посчитаем R2 score.

In [15]:
model_t = LinReg()
model_t.fit(X_train_t, y_train)
predictions = model_t.predict(X_test_t)
print(r2_score(y_test, predictions))

0.4117683571261317


Как видим, результат тот же: **качество модели, обученной на преобразованных данных, не изменилось.**

(Разница есть в 7-9 знаке после запятой, ею можно пренебречь. Это погрешность при операциях с данными типа float.)

## Выводы

- Для защиты персональных данных клиентов страховой компании использовали преобразование данных методом умножения исходной матрицы признаков на произвольную обратимую матрицу.


- Матрица, которую используем для кодирования, должна удовлетворять следующим требованиям: 1) быть квадратной и обратимой 2) иметь столько же строк / столбцов, сколько столбцов в исходной матрице.


- Выяснили, что при использовании преобразованных таким способом данных параметры модели будут отличаться, но итоговые предсказания получатся такими же, как без преобразования данных. А значит, качество модели не изменится.


- Выяснили, как связаны параметры модели в случае использования исходных и преобразованных данных:
$$
w_p = P^{-1} w
$$
Параметры модели при использовании преобразованных данных можно рассчитать как скалярное произведение матрицы, обратной к матрице кодирования P, и вектора параметров при использовании исходных данных.


- Доказали, что предсказания в обоих случаях будут одинаковыми:
$$
a_p = XPw_p = XPP^{-1}w = Xw = a
$$

- Разработали алгоритм, который принимает исходные данные, преобразовывает их методом умножения на обратимую матрицу соответствующего размера, обучает модель на преобразованных данных и выдает предсказания.


- Проверили работу алгоритма и убедились, что качество модели не меняется.